In [22]:
import pandas as pd
pd.options.mode.chained_assignment = None
import os
import glob
from os import path
import numpy as np
from datetime import datetime, timedelta
import matplotlib
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
import scipy.stats

In [25]:
cohort_df = "/home/local/VANDERBILT/litz/github/MASILab/DeepLungScreening/cohorts/livu/livu_prep_cs.csv"
cohort_df = pd.read_csv(cohort_df)
cohort_df

,pid,scan_date,filename,session,LC_CODE,label_date,scan_label,lung_cancer,bmi,age,sex,race,id
0,66698488,2017-01-25,66698488time20170125,1,C78.01,2019-03-01,2,1,28.09,58.0,0.0,1.0,66698488time20170125
1,390776127,2018-05-16,390776127time20180516,0,NaN,2018-12-17,3,0,14.92,36.0,1.0,1.0,390776127time20180516
2,476264702,2015-03-31,476264702time20150331,0,NaN,2015-03-31,3,0,29.54,59.0,1.0,1.0,476264702time20150331
3,553520524,2016-12-16,553520524time20161216,0,162.9,2017-02-15,3,1,26.04,55.0,1.0,2.0,553520524time20161216
4,557278028,2018-01-27,557278028time20180127,1,C34.02,2018-03-01,3,1,22.71,61.0,0.0,1.0,557278028time20180127
...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,41401448383,2016-09-21,41401448383time20160921,1,162.4,2017-01-18,3,1,25.58,72.0,0.0,2.0,41401448383time20160921
157,42046041233,2016-02-23,42046041233time20160223,0,NaN,2016-02-29,3,0,29.70,64.0,1.0,1.0,42046041233time20160223
158,42215796359,2015-03-19,42215796359time20150319,0,NaN,2015-03-19,3,0,29.15,51.0,0.0,1.0,42215796359time20150319
159,42477606763,2016-05-11,42477606763time20160511,0,NaN,2016-08-30,3,0,16.12,80.0,0.0,1.0,42477606763time20160511


### DL model results

In [26]:
cscnn = '/home/local/VANDERBILT/litz/github/MASILab/DSB2017/models/config_livu_1206/preds.csv'
cscnn = pd.read_csv(cscnn)
cscnn['cscnn_risk'] = cscnn['pred']
cscnn['id'] = cscnn['id'].apply(lambda x: x.split("'")[1])
cohort_df = cohort_df.merge(cscnn[['id', 'cscnn_risk']], on='id')

In [27]:
dlstm_path = "/home/local/VANDERBILT/litz/github/MASILab/RNNLung/compare/tumor_NLST/models/1206_livu/pred.csv"
dlstm = pd.read_csv(dlstm_path)
dlstm['dlstm_risk'] = dlstm['pred']
cohort_df = cohort_df.merge(dlstm[['pid', 'dlstm_risk']], on='pid', how='left')

In [28]:
tdvit_path = "/home/local/VANDERBILT/litz/github/MASILab/time-distance-transformer/models/1207_livu_td/pred.csv"
tdvit = pd.read_csv(tdvit_path)
tdvit['tdvit_risk'] = tdvit['pred']
cohort_df = cohort_df.merge(tdvit[['PID', 'tdvit_risk']], left_on='pid', right_on='PID', how='left')

In [29]:
cohort_df

,pid,scan_date,filename,session,LC_CODE,label_date,scan_label,lung_cancer,bmi,age,sex,race,id,cscnn_risk,dlstm_risk,PID,tdvit_risk
0,66698488,2017-01-25,66698488time20170125,1,C78.01,2019-03-01,2,1,28.09,58.0,0.0,1.0,66698488time20170125,0.510093,0.326652,6.669849e+07,0.432084
1,390776127,2018-05-16,390776127time20180516,0,NaN,2018-12-17,3,0,14.92,36.0,1.0,1.0,390776127time20180516,0.960203,NaN,NaN,NaN
2,476264702,2015-03-31,476264702time20150331,0,NaN,2015-03-31,3,0,29.54,59.0,1.0,1.0,476264702time20150331,0.211456,NaN,NaN,NaN
3,553520524,2016-12-16,553520524time20161216,0,162.9,2017-02-15,3,1,26.04,55.0,1.0,2.0,553520524time20161216,0.852679,NaN,NaN,NaN
4,557278028,2018-01-27,557278028time20180127,1,C34.02,2018-03-01,3,1,22.71,61.0,0.0,1.0,557278028time20180127,0.917059,0.609782,5.572780e+08,0.791639
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,41401448383,2016-09-21,41401448383time20160921,1,162.4,2017-01-18,3,1,25.58,72.0,0.0,2.0,41401448383time20160921,0.150413,0.392314,4.140145e+10,0.362872
157,42046041233,2016-02-23,42046041233time20160223,0,NaN,2016-02-29,3,0,29.70,64.0,1.0,1.0,42046041233time20160223,0.945740,NaN,NaN,NaN
158,42215796359,2015-03-19,42215796359time20150319,0,NaN,2015-03-19,3,0,29.15,51.0,0.0,1.0,42215796359time20150319,0.983303,NaN,NaN,NaN
159,42477606763,2016-05-11,42477606763time20160511,0,NaN,2016-08-30,3,0,16.12,80.0,0.0,1.0,42477606763time20160511,0.060721,NaN,NaN,NaN


### Evaluate cross sectional results

In [30]:
def compute_model_auc(sample, model):
    y = sample['lung_cancer']
    y_prob = sample[f"{model}_risk"]
    return roc_auc_score(y, y_prob)

def compute_ci(data, confidence=0.95):
    a = 1.0*np.array(data)
    n = len(a)
    mu, se = np.mean(a), scipy.stats.sem(a)
    h = se*scipy.stats.t.ppf((1+confidence)/2.0, n-1)
    # print(mu)
    return mu, mu-h, mu+h

In [31]:
random_seed=56
n_bootstrap = 1000 # number of bootstrap samples with replacement (the samples are same size as cohort)

model_names = ['cscnn']

nonnull_cohort = cohort_df[~cohort_df['cscnn_risk'].isnull()]

dfrows= []
for model in model_names:
    # calculate 95% CI with bootstrap sampling
    aucs = []
    for i in range(n_bootstrap):
        sample = nonnull_cohort.sample(frac=1.0, replace=True)
        aucs.append(compute_model_auc(sample, model))

    mean_auc, ci_low, ci_high = compute_ci(aucs)
    dfrows.append({'model': model, 'mean_AUC': mean_auc, 'ci_low':ci_low, 'ci_high':ci_high, 'std': np.std(aucs)})

metrics = pd.DataFrame(dfrows)
metrics

,model,mean_AUC,ci_low,ci_high,std
0,cscnn,0.689446,0.686613,0.69228,0.045644


### Eval longitudinal models

In [32]:
nonnull_cohort = cohort_df[~cohort_df['tdvit_risk'].isnull() & ~cohort_df['dlstm_risk'].isnull()]

model_names = ['tdvit', 'dlstm']

dfrows= []
for model in model_names:
    # calculate 95% CI with bootstrap sampling
    aucs = []
    for i in range(n_bootstrap):
        sample = nonnull_cohort.sample(frac=1.0, replace=True)
        aucs.append(compute_model_auc(sample, model))

    mean_auc, ci_low, ci_high = compute_ci(aucs)
    dfrows.append({'model': model, 'mean_AUC': mean_auc, 'ci_low':ci_low, 'ci_high':ci_high, 'std':np.std(aucs)})

metrics = pd.DataFrame(dfrows)
metrics

,model,mean_AUC,ci_low,ci_high,std
0,tdvit,0.839817,0.83611,0.843524,0.059705
1,dlstm,0.741527,0.73693,0.746123,0.074033
